# Обучение с учителем для решения задач коммивояжера


In [1]:
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Скрывает INFO-логи (оставляет WARNING и ERROR)
tf.get_logger().setLevel('ERROR')  # Скрывает большинство логов TensorFlow
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)

import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

2025-04-09 10:24:32.273951: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744194272.339325    1632 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744194272.354116    1632 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744194272.454527    1632 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744194272.454541    1632 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744194272.454543    1632 computation_placer.cc:177] computation placer alr

In [2]:
import numpy as np
from tensorflow.keras.layers import Input, Dense, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tqdm import tqdm
from python_tsp.exact import solve_tsp_dynamic_programming
from sklearn.metrics import mean_absolute_percentage_error

### Модель предсказывает какие дуги входят в маршрут, но не их последовательность

In [180]:
class TSPSolver:
    def __init__(self, num_cities, hidden_dim=256):
        self.num_cities = num_cities
        self.hidden_dim = hidden_dim
        self.model = self._build_model()
    
    def _build_model(self):
        # Вход: матрица расстояний (batch, cities, cities)
        inputs = Input(shape=(self.num_cities, self.num_cities))
        
        # Кодировщик на основе полносвязных слоев
        x = Dense(self.hidden_dim, activation='relu')(inputs)
        x = LayerNormalization()(x)
        x = Dense(self.hidden_dim, activation='relu')(x)
        x = LayerNormalization()(x)

        x = Dense(self.hidden_dim, activation='relu')(x)
        x = LayerNormalization()(x)
        
        # Выходной слой - вероятности переходов
        logits = Dense(self.num_cities)(x)
        outputs = tf.keras.activations.softmax(logits)
        
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer=Adam(0.001), loss=self._custom_loss)
        return model
    
    def _custom_loss(self, y_true, y_pred):
        # y_true: маска посещенных городов (batch, cities, cities)
        # y_pred: вероятности переходов (batch, cities, cities)
        
        # Применяем маску к предсказаниям
        masked_pred = y_pred * y_true
        
        # Нормализуем вероятности
        masked_pred = masked_pred / (K.sum(masked_pred, axis=-1, keepdims=True) + K.epsilon())
        
        # Вычисляем кросс-энтропию
        loss = -K.sum(y_true * K.log(masked_pred + K.epsilon()), axis=-1)
        return K.mean(loss)
    
    def train(self, X_train, routes, epochs=50, batch_size=128):
        """
        X_train: матрицы расстояний (samples, cities, cities)
        routes: оптимальные маршруты (samples, cities)
        """
        # Создаем маски переходов для обучения
        y_masks = np.zeros_like(X_train)
        
        for i, route in enumerate(routes):
            for j in range(len(route)-1):
                from_city = route[j]
                to_city = route[j+1]
                y_masks[i, from_city, to_city] = 1
        
        self.model.fit(
            X_train,
            y_masks,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=0.1
        )

In [3]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


### Подготовка исходных данных и поиск точного решения методом динамического программирования занимает час

In [10]:
N = 12
nlen = N*N
rand = np.random.RandomState(1)

In [ ]:
cnt = 50000
lx = []
ly = []
ld = []
for i in tqdm(range(cnt)):
    a = rand.normal(20, 5, size=(N, N))
    #a = np.round((a + a.T)/2,2)
    a[np.eye(N)==1] = 0
    permutation, distance = solve_tsp_dynamic_programming(a)
    lx.append(a)
    ly.append(permutation)
    ld.append(distance)
X_train = np.array(lx)
Y_train = np.array(ly)
D_train = np.array(ld)

In [62]:
#np.save('X_train', X_train)
#np.save('Y_train', Y_train)
X_train = np.load('X_train.npy')
Y_train = np.load('Y_train.npy')


### Инициализируем и обучаем модель

In [181]:
solver = TSPSolver(num_cities=N)
solver.train(X_train, Y_train, epochs=40)

Epoch 1/40
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 1.0552e-06 - val_loss: 6.5155e-07
Epoch 2/40
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6.1037e-07 - val_loss: 4.9647e-07
Epoch 3/40
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4.6749e-07 - val_loss: 4.0588e-07
Epoch 4/40
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.9230e-07 - val_loss: 3.6841e-07
Epoch 5/40
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.6084e-07 - val_loss: 3.5061e-07
Epoch 6/40
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.4573e-07 - val_loss: 3.4328e-07
Epoch 7/40
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.3793e-07 - val_loss: 3.3438e-07
Epoch 8/40
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.3204e-07 - val_loss: 3.3095e-07
Epoch 9/40
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.2887e-07 - val_loss: 3.3150e-07
Epoch 10/40
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.2493e-07 - val_loss: 3.2483e-07
Epoch 11/40
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss

### Проверка обученной модели

In [289]:
def predict_route(solver, dist_matrix, is_rnd = False):
    ap = solver.model.predict(dist_matrix[np.newaxis, ...], verbose=0)[0]
    num_cities = dist_matrix.shape[0]
    best_route = []
    best_dist = np.inf
    for _ in range(500):
        p = np.max(ap, axis=1)
        if is_rnd: p = np.full_like(p, 1)
        p = p / np.sum(p)
        current = rand.choice(range(num_cities), p=p)
        visited = {current}
        route = [current]
        total_dist = 0
        for _ in range(num_cities-1):
            probs = ap[current].copy()
            if is_rnd: probs = np.full_like(probs, 1) 
            # Маскируем посещенные города
            probs[list(visited)] = 0
            # Выбираем следующий город
            next_city = rand.choice(range(num_cities), p=(probs / np.sum(probs)))
            # next_city = np.argmax(probs)
            route.append(next_city)
            visited.add(next_city)
            total_dist += dist_matrix[current, next_city]
            current = next_city
        total_dist += dist_matrix[route[-1], route[0]]
        if total_dist < best_dist:
            best_dist = total_dist
            best_route = route
    return np.array(best_route)

In [290]:
cnt = 50
ld = []
lp = []
lr = []
for i in tqdm(range(cnt)):
    a = rand.normal(20, 5, size=(N, N))
    # a = np.round((a + a.T)/2,2)
    a[np.eye(N)==1] = 0
    permutation, distance = solve_tsp_dynamic_programming(a)
    route = predict_route(solver, a)
    total_dist = sum(a[route[i], route[i+1]] for i in range(N-1))
    total_dist += a[route[-1], route[0]]
    ld.append(distance)
    lp.append(total_dist)
Y_predict = np.array(lp)
Y_true = np.array(ld)

100%|██████████| 50/50 [00:19<00:00,  2.52it/s]


In [291]:
mean_absolute_percentage_error(Y_true, Y_predict)

0.052357125615434645

In [292]:
# Сколько случаев действительно плохого прогноза
sum(((Y_predict - Y_true) / Y_true) > 0.2)

np.int64(0)